<a href="https://colab.research.google.com/github/C10udburst/ml-playground/blob/master/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will create a book recommendation algorithm using **K-Nearest Neighbors**.

You will use the [Book-Crossings dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/). This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users. 

After importing and cleaning the data, use `NearestNeighbors` from `sklearn.neighbors` to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures distance to determine the “closeness” of instances.

Create a function named `get_recommends` that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:

`get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")`

should return:

```
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301], 
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]
```

Notice that the data returned from `get_recommends()` is a list. The first element in the list is the book title passed in to the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed in to the function.

If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

The first three cells import libraries you may need and the data to use. The final cell is for testing. Write all your code in between those cells.

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

--2022-06-24 07:04:38--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  11.2MB/s    in 2.2s    

2022-06-24 07:04:41 (11.2 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [ ]:
# get data files

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
df_ratings.head()

,user,isbn,rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0


In [ ]:
df_books.head()

,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber


In [ ]:
df = df_ratings.merge(df_books, left_on='isbn', right_on='isbn', how='inner')
df = df.drop('author', axis=1)
df.head()

,user,isbn,rating,title
0,276725,034545104X,0.0,Flesh Tones: A Novel
1,2313,034545104X,5.0,Flesh Tones: A Novel
2,6543,034545104X,0.0,Flesh Tones: A Novel
3,8680,034545104X,5.0,Flesh Tones: A Novel
4,10314,034545104X,9.0,Flesh Tones: A Novel


In [ ]:
book_counts = df \
  .groupby(['isbn']) \
  ['rating'] \
  .count() \
  .reset_index() \
  .rename(columns={"rating": "ratingCountBook"})
book_counts.head()

,isbn,ratingCountBook
0,0000913154,1
1,0001010565,2
2,0001046438,1
3,0001046713,1
4,000104687X,1


In [ ]:
user_counts = df \
  .groupby(['user']) \
  ['rating'] \
  .count() \
  .reset_index() \
  .rename(columns={"rating": "ratingCountUser"})
user_counts.head()

,user,ratingCountUser
0,2,1
1,8,17
2,9,3
3,10,1
4,12,1


In [ ]:
df = df.merge(book_counts, left_on='isbn', right_on='isbn', how='inner')
df = df.merge(user_counts, left_on='user', right_on='user', how='inner')
df.head()

,user,isbn,rating,title,ratingCountBook,ratingCountUser
0,276725,034545104X,0.0,Flesh Tones: A Novel,60,1
1,2313,034545104X,5.0,Flesh Tones: A Novel,60,36
2,2313,0812533550,9.0,Ender's Game (Ender Wiggins Saga (Paperback)),23,36
3,2313,0679745580,8.0,In Cold Blood (Vintage International),55,36
4,2313,0060173289,9.0,Divine Secrets of the Ya-Ya Sisterhood : A Novel,130,36


In [ ]:
# remove all users with < 100 and books with < 200
df = df.query('ratingCountBook >= 200 & ratingCountUser >= 100')
df = df.drop(['ratingCountBook', 'ratingCountUser'], axis=1)
df.head()

,user,isbn,rating,title
38,6543,0316666343,0.0,The Lovely Bones: A Novel
39,6543,0385504209,0.0,The Da Vinci Code
40,6543,0971880107,0.0,Wild Animus
41,6543,0312966970,0.0,Four To Score (A Stephanie Plum Novel)
43,6543,0446605484,10.0,Roses Are Red (Alex Cross Novels)


In [ ]:
df_cross = df.pivot_table(index='title', columns='user', values='rating', aggfunc='mean', fill_value=0)
df_cross.head()

user,254,507,882,1424,1435,1733,1903,2033,2110,2276,...,275020,275970,276463,276680,277427,277478,277639,278137,278188,278418
title,,,,,,,,,,,,,,,,,,,,,
1st to Die: A Novel,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2nd Chance,0,0,0,0,0,0,0,0,0,10,...,0,0,0,0,0,0,0,0,0,0
A Bend in the Road,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"A Child Called \It\"": One Child's Courage to Survive""",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A Heartbreaking Work of Staggering Genius,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_cross.shape

(192, 1588)

In [149]:
book_knn = NearestNeighbors(
    radius='0.7',
    n_neighbors=5)
book_knn.fit(df_cross)

NearestNeighbors(radius='0.7')

In [150]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  recommended_books = [book, []]
  key = list(df_cross.index).index(book)
  distances, indices = book_knn.kneighbors(df_cross.iloc[key, :].values.reshape(1, -1), n_neighbors=6)
  for dist, idx in zip(distances.flatten(), indices.flatten()):
    if idx == key:
      continue
    recommended_books[1].append([
      df_cross.index[idx], dist
    ])

  return recommended_books

In [ ]:
get_recommends("A Heartbreaking Work of Staggering Genius")

Use the cell below to test your function. The `test_book_recommendation()` function will inform you if you passed the challenge or need to keep trying.

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["Songs in Ordinary Time (Oprah's Book Club (Paperback))", 81.21576201698781], ['The Last Time They Met : A Novel', 81.55366331440912], ['Here on Earth', 81.61494961096282], ['Digital Fortress : A Thriller', 82.60750571225353], ['How to Be Good', 82.76472678623425]]]
You haven't passed yet. Keep trying!
